In [ ]:
import csv
import numpy as np
import open3d as o3d
from matplotlib import pyplot as plt 
from sklearn.neighbors import KDTree
from sklearn.decomposition import PCA
import plotly.graph_objects as go
import ipywidgets as widgets # for interactive sliders
import pickle
from timeit import timeit
from scipy.spatial import cKDTree

from itertools import product


%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
from digiforest_analysis.utils.timing import Timer
timer = Timer()

In [ ]:
DATASET_DIR = "/home/ori/git/realtime-trees/single_trees/clustering_2/"
SLICE_HEIGHT = 0.5
TREE_ID = 63

In [ ]:
file_name = DATASET_DIR + "tree" + str(TREE_ID).zfill(3) + ".pkl"
with open(file_name, 'rb') as file:
    cluster = pickle.load(file)

In [ ]:
slice_height = 2.0
slice_thickness = 2.0
cloud = cluster['cloud'].point.positions.numpy()
slice = cloud[np.logical_and(cloud[:, 2] >= slice_height - slice_thickness / 2, cloud[:, 2] < slice_height + slice_thickness / 2,)][:, :2]
print(len(slice))

# timing_hough = timeit("Circle.from_cloud_hough(slice)", "from __main__ import Circle, slice", number=1000)
# print(f"Hough algo took {timing_hough:.3f} ms")
# timing_new = timeit("new_hough(slice)", "from __main__ import new_hough, slice", number=1000)
# print(f"New algo took {timing_new:.3f} ms")

with timer("OLD"):
    circ_hough = Circle.from_cloud_hough(slice)
with timer("NEW"):
    # circ_new, circles = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_points=100)
    circ_new = Circle.from_cloud_ransahc(slice, min_radius=0.05, max_radius=0.5, sampling="weighted", max_circles=500)

print(timer)
plt.scatter(slice[:, 0], slice[:, 1], s=5)
plt.gca().set_aspect('equal', adjustable='box')
# plot circle
plt.gca().add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False))
plt.gca().add_artist(plt.Circle((circ_new.x, circ_new.y), circ_new.radius, color='g', fill=False))
# plt.scatter(circles[:, 0], circles[:, 1], s=5, color='r')

In [ ]:
import colorsys
import os
import pickle

import numpy as np
from matplotlib import pyplot as plt
import ipywidgets as widgets
%load_ext autoreload 
%autoreload 2
from digiforest_analysis.tasks.tree_reconstruction import Tree, Circle
# %matplotlib notebook

def align_clouds(clusters):
    trafos_map = [c["info"]["T_sensor2map"] @ c["info"]["axis"]["transform"] for c in clusters]
    clouds_map = [c["cloud"].point.positions.numpy() @ c["info"]["T_sensor2map"][:3, :3].T + c["info"]["T_sensor2map"][:3, 3] for c in clusters]
    mean_position = np.mean([t[:3, 3] for t in trafos_map], axis=0)
    deltas = [t[:3, 3] - mean_position for t in trafos_map]
    deltas = [np.array([d[0], d[1], 0]) for d in deltas]
    return [c - d for c, d in zip(clouds_map, deltas)]

path = "/home/ori/git/digiforest_drs/trees/logs/raw"
TREE_ID = 19
tree_path = os.path.join(path, f"tree{TREE_ID:03d}.pkl")
with open(tree_path, 'rb') as file:
    tree : Tree = pickle.load(file)

# # make 3d plot of clusters in different colors
# fig = go.Figure()
# for cluster in tree.clusters:
#     cloud = cluster['cloud'].point.positions.numpy()
#     transform = cluster['info']['T_sensor2map']
#     cloud = cloud @ transform[:3, :3].T + transform[:3, 3]
#     fig.add_trace(go.Scatter3d(x=cloud[:, 0], y=cloud[:, 1], z=cloud[:, 2], mode='markers', marker=dict(size=1)))
# fig.show()

# make a 2d plot with a slider that changes the height of the slice
clouds = [
    cluster['cloud'].point.positions.numpy() @ cluster['info']['T_sensor2map'][:3, :3].T + cluster['info']['T_sensor2map'][:3, 3]
    for cluster in tree.clusters
]
clouds_aligned = align_clouds(tree.clusters)
cloud_merged = np.concatenate(clouds, axis=0)
cloud_aligned_merged = np.concatenate(clouds_aligned, axis=0)
# cloud_aligned_merged = tree.points
hues = np.linspace(0, 1, len(clouds) + 1)[:len(clouds)]
colors = [colorsys.hsv_to_rgb(h, 1, 1) for h in hues]

slice_thickness = 0.2
# min and max width and height are 10th and 90 th percentiles
min_width, max_width = np.percentile(cloud_merged[:, 0], [2, 98])
min_height, max_height = np.percentile(cloud_merged[:, 1], [2, 98])
@widgets.interact(height=widgets.FloatSlider(min=cloud_merged[:, 2].min(), max=cloud_merged[:, 2].max(), step=0.3, value=0.0))
def update_slice_height(height):
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    for a in ax:
        a.set_aspect('equal', adjustable='box')
        a.set_xlim(min_width, max_width)
        a.set_ylim(min_height, max_height)
    for cloud, color in zip(clouds, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[0].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    for cloud, color in zip(clouds_aligned, colors):
        slice = cloud[np.logical_and(cloud[:, 2] >= height - slice_thickness / 2, cloud[:, 2] < height + slice_thickness / 2,)][:, :2]
        ax[1].scatter(slice[:, 0], slice[:, 1], s=1, color=color)
    slice_merged = cloud_merged[np.logical_and(cloud_merged[:, 2] >= height - slice_thickness / 2, cloud_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    slice_aligned_merged = cloud_aligned_merged[np.logical_and(cloud_aligned_merged[:, 2] >= height - slice_thickness / 2, cloud_aligned_merged[:, 2] < height + slice_thickness / 2,)][:, :2]
    ax[2].scatter(slice_aligned_merged[:, 0], slice_aligned_merged[:, 1], s=1, color='k')
    # fit hough circle
    center_region = Circle(tree.axis["transform"][:3, 3], tree.axis["radius"])
    min_radius = 0.5 * tree.axis["radius"]
    max_radius = 1.5 * tree.axis["radius"]
    circ_hough = Circle.from_cloud_hough(slice_merged, min_radius=0.05, max_radius=0.5)
    circ_hough_aligned = Circle.from_cloud_hough(slice_aligned_merged, min_radius=0.05, max_radius=0.5)
    circ_ransahc = Circle.from_cloud_ransahc(slice_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    circ_ransahc_aligned = Circle.from_cloud_ransahc(slice_aligned_merged, min_radius=min_radius, max_radius=max_radius, center_region=center_region)
    ax[0].add_artist(plt.Circle((circ_hough.x, circ_hough.y), circ_hough.radius, color='r', fill=False, linewidth=2))
    ax[0].add_artist(plt.Circle((circ_ransahc.x, circ_ransahc.y), circ_ransahc.radius, color='g', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_hough_aligned.x, circ_hough_aligned.y), circ_hough_aligned.radius, color='r', fill=False, linewidth=2))
    ax[2].add_artist(plt.Circle((circ_ransahc_aligned.x, circ_ransahc_aligned.y), circ_ransahc_aligned.radius, color='g', fill=False, linewidth=2))

In [ ]:
import trimesh
import os
dir = 'single_trees/clustering/'
for file in os.listdir(dir):
    if "pcd" not in file:
        continue
    points = np.asarray(o3d.io.read_point_cloud(os.path.join(dir, file)).points)
    tree = Tree.reconstruct(
        points,
        SLICE_HEIGHT,
        slice_thickness=0.2,
        save_points=True,
        save_debug_results=True,
        save_votes=True
    )
    verts, tris = tree.generate_mesh()
    if tris.shape[0] == 0:
        continue
    mesh = trimesh.Trimesh(vertices=verts, faces=tris)
    mesh.export(os.path.join('single_trees/clustering_map/meshes', file.replace('.pcd', '.obj')))